In [ ]:
import sys
sys.path.append('../../apps/SONIA')
sys.path.append('../../apps/OLGA')
sys.path.append('../../apps/soNNia')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from os.path import exists

%matplotlib inline

In [ ]:
from sonnia.sonnia import SoNNia

In [ ]:
# settings
pep = "NLVPMVATV"

In [ ]:
# load and prepare emerson sequences for SONIA
train_seqs_df = pd.read_csv('./train_train_WithoutDuplicates_aligned_20.csv.gz')
t_seqs = train_seqs_df['amino_acid'].to_list()
t_v = train_seqs_df['v_gene'].to_list()
t_j = train_seqs_df['j_gene'].to_list()
sonia_input_emerson = [list(a) for a in zip(t_seqs, t_v, t_j)]

# select subset of 10^6 seqs
n_max = 1000000
raninds = np.arange(len(sonia_input_emerson))
rng = np.random.default_rng(2021)
rng.shuffle(raninds)
raninds = raninds[:n_max]
sonia_input_emerson_1e6 = list(np.array(sonia_input_emerson)[raninds])

In [ ]:
# load and prepare peptide-specific seqs for SONIA
vdgdb_df = pd.read_csv('./' + pep + '/VDJdb_' + pep + '_WithAligned20.csv')
vdgdb_df = vdgdb_df.drop_duplicates().reset_index(drop=True)
t_seqs = vdgdb_df['CDR3_beta'].to_list()
t_v = vdgdb_df['TRBV_gene'].to_list()
t_j = vdgdb_df['TRBJ_gene'].to_list()
sonia_input_vdgdb = [list(a) for a in zip(t_seqs, t_v, t_j)]

In [ ]:
# load and prepare second set of Emerson seqs for SONIA (to be used as negative)
filename_cdr3raw = './train_data_1.txt' 
inds_non_overlap = np.loadtxt('./1_inds_nonoverlap_0.txt').astype(np.int16)
t_seq0 = []
with open(filename_cdr3raw) as f:
    for line in f:
        linesplit = line.strip().split('\n')
        t_seq0.append(linesplit[0])

t_seq = [x.split('\t') for x in np.array(t_seq0)[inds_non_overlap]]

In [ ]:
len(sonia_input_emerson_1e6) / int(80*len(sonia_input_vdgdb)/100)

274.8763056624519

In [ ]:
# settings2 
l2_fin = 0
epo = 30
bs = 10000

In [ ]:
# main computation: AUROCs

if exists('./' + pep + '/AUROCs.txt'):
    AUROCs = np.loadtxt('./' + pep + '/AUROCs.txt')
else:
    AUROCs = []
for i in range(50):
    repl = i
    
    ## prepare positives (train, test) ##
    path_o ='./' + pep + '/indices/index_permutation_repl' + str(repl) + '.txt'
    full_intR = (np.loadtxt(path_o)).astype(np.int16)
    data = [sonia_input_vdgdb[t] for t in full_intR]
    train_data = data[:int(80*len(data)/100)]
    val_data = data[int(80*len(data)/100):]

    ## prepare negatives (test) ##
    path_o ='./' + pep + '/indices/index_permutationN_repl' + str(repl) + '.txt'
    full_intR = (np.loadtxt(path_o)).astype(np.int16)
    val_dataN0 = [t_seq[t] for t in full_intR]
    val_dataN = val_dataN0[:len(val_data)]
    
    ## train model ##
    qm = SoNNia(data_seqs = train_data, 
            gen_seqs = sonia_input_emerson_1e6,
            l2_reg = l2_fin,
            deep=False, include_joint_genes=True, include_indep_genes=False,
            )
    qm.infer_selection(epochs = epo, batch_size = bs, validation_split=0.01, verbose=0)
    
    ## check for nans ##
    t_min = np.min(qm.likelihood_train)
    if np.isnan(t_min):
        print("ERROR: nan obtained! Try to have a larger minibatch to prevent this...")
        AUROCs = np.append(AUROCs, append(-1))
    
    ## compute AUROC ##
    LR_vdgdbn = [qm.find_seq_features(x) for x in val_data]
    LR_emerson = [qm.find_seq_features(x) for x in val_dataN]
    scores_positive = - qm.compute_energy(LR_vdgdbn)
    scores_negative = - qm.compute_energy(LR_emerson)    
    labels = np.hstack((np.zeros((len(scores_negative))), np.ones((len(scores_positive))))) 
    scores = np.hstack((scores_negative, scores_positive))
    fpr, tpr, thresholds = metrics.roc_curve(labels, scores)
    AUROCs = np.append(AUROCs, metrics.auc(fpr, tpr))
    
    # save resulting AUROC file (this rewrites the file completely, but it is a short file so no problem...)
    np.savetxt('./' + pep + '/AUROCs.txt', AUROCs)
    
    # save resulting positives_scores
    np.savetxt('./' + pep + '/scores_positives_' + str(i) + '.txt', scores_positive)
    
    # save resulting negatives_scores
    np.savetxt('./' + pep + '/scores_negatives_' + str(i) + '.txt', scores_negative)

 44%|████▍     | 1601/3638 [00:00<00:00, 16007.57it/s]

Encode data.


  0%|          | 2693/1000000 [00:00<01:14, 13459.21it/s]

Encode gen.


 18%|█▊        | 654/3638 [00:00<00:00, 6535.24it/s]

...

 33%|███▎      | 1209/3638 [00:00<00:00, 12087.10it/s]

Encode data.


  0%|          | 1354/1000000 [00:00<01:13, 13538.43it/s]

Encode gen.


100%|██████████| 1000000/1000000 [01:26<00:00, 11625.54it/s]
